In [28]:
import cudf
import cupy as cp
import plotly.graph_objects as go
import datashader as ds
import colorcet
import os

# Visualisation Imports
import numpy as np
import xarray as xr
# datashader
import datashader as ds
import datashader.transfer_functions as tf
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.transfer_functions import Images, Image
from datashader.colors import Elevation
from datashader.utils import orient_array

# holoviews
import holoviews as hv
from holoviews.plotting.plotly.dash import to_dash
from holoviews.element.tiles import CartoDark
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
from holoviews.operation import decimate

# plotly
from plotly.colors import sequential
from plotly.subplots import make_subplots

# Dash Import
import dash
import dash_html_components as html
from jupyter_dash import JupyterDash

#statsmodels
import statsmodels as sm
from statsmodels.regression import linear_model
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

In [2]:
DATA_PATH = '../../data/hycom'
RES_PATH = '../../results/hycom'

df = cudf.read_csv(os.path.join(RES_PATH, 'hycom_equinox_merged-201920.csv'))

# filter to only valid measurements
df = df[df['WOCE_QC_FLAG'] == 2]
df.head()

,start_date,lat,lon,water_temp_0,salinity_0,water_temp_2,salinity_2,water_temp_4,salinity_4,water_temp_6,...,xCO2_ATM_interpolated_ppm,PRES_EQU_hPa,PRES_ATM@SSP_hPa,TEMP_EQU_C,SST_C,SAL_permil,fCO2_SW@SST_uatm,fCO2_ATM_interpolated_uatm,dfCO2_uatm,WOCE_QC_FLAG
0,2019-11-19,19.0418,-87.4192,28.641375,36.191375,28.48175,36.194000,28.490375,36.193750,28.494625,...,410.81,1014.72,1013.5662,29.14,29.1692,36.2848,410.02,393.66,16.36,2.0
1,2020-01-21,18.3305,-65.1468,26.471600,35.826667,26.42980,35.825867,26.429333,35.828200,26.426533,...,417.55,1015.40,1014.5055,27.25,27.3215,35.3726,390.25,402.13,-11.88,2.0
2,2019-07-31,19.7638,-87.1375,29.324125,36.082625,29.24825,36.083875,29.235125,36.083500,29.218875,...,410.89,1014.90,1014.5565,29.98,29.7258,36.2400,435.94,393.63,42.32,2.0
3,2019-09-13,25.9173,-79.9605,29.782625,36.122750,29.79925,36.123500,29.802750,36.124125,29.803750,...,414.53,1012.47,1011.9442,29.88,29.9378,35.4666,413.24,395.84,17.40,2.0
4,2019-08-11,23.2150,-83.5792,30.861500,36.154125,30.54200,36.152875,30.412500,36.150375,30.245500,...,417.82,1017.40,1017.0594,30.57,30.6395,36.3900,442.06,400.40,41.66,2.0



## Exploring Interaction between Sentinel (Satellite Data) on ground-truth labels (Equinox)

Dropping date, lat and lon since these are (from literature) expected to induce multi-collinearity

In [3]:
df.drop(['start_date', 'lat', 'lon', 'WOCE_QC_FLAG', 'easting', 'northing'], axis=1, inplace=True)
df.head()

,water_temp_0,salinity_0,water_temp_2,salinity_2,water_temp_4,salinity_4,water_temp_6,salinity_6,water_temp_8,salinity_8,xCO2_EQU_ppm,xCO2_ATM_interpolated_ppm,PRES_EQU_hPa,PRES_ATM@SSP_hPa,TEMP_EQU_C,SST_C,SAL_permil,fCO2_SW@SST_uatm,fCO2_ATM_interpolated_uatm,dfCO2_uatm
0,28.641375,36.191375,28.48175,36.194000,28.490375,36.193750,28.494625,36.193500,28.495625,36.193250,426.816,410.81,1014.72,1013.5662,29.14,29.1692,36.2848,410.02,393.66,16.36
1,26.471600,35.826667,26.42980,35.825867,26.429333,35.828200,26.426533,35.830333,26.422667,35.832533,403.554,417.55,1015.40,1014.5055,27.25,27.3215,35.3726,390.25,402.13,-11.88
2,29.324125,36.082625,29.24825,36.083875,29.235125,36.083500,29.218875,36.083000,29.198750,36.082250,460.100,410.89,1014.90,1014.5565,29.98,29.7258,36.2400,435.94,393.63,42.32
3,29.782625,36.122750,29.79925,36.123500,29.802750,36.124125,29.803750,36.124375,29.803375,36.124375,431.397,414.53,1012.47,1011.9442,29.88,29.9378,35.4666,413.24,395.84,17.40
4,30.861500,36.154125,30.54200,36.152875,30.412500,36.150375,30.245500,36.148625,29.899750,36.150000,459.698,417.82,1017.40,1017.0594,30.57,30.6395,36.3900,442.06,400.40,41.66


### Interaction Model Printing Funcitons

In [102]:
X_col_names = ['water_temp_0', 'salinity_0', 'water_temp_2', 'salinity_2', 'water_temp_4', 'salinity_4', 'water_temp_6', 'salinity_6', 'water_temp_8', 'salinity_8']


def fit_model(df, pred_col, limit=True):
    
    df = df.dropna()
    
    X = df[X_col_names].to_pandas()
    y = df[[pred_col]].to_pandas()
    
    X = sm.tools.tools.add_constant(X)
    model = linear_model.OLS(y, X).fit()
    if limit:
        model = model.pvalues[model.pvalues <= 0.05]
        return model.sort_values()
    else:
        return model
    
def fit_interaction_model(df, pred_col, limit=True):
    
    df = df.dropna()
    
    X = df[X_col_names]
    y = df[[pred_col]]
    poly_features = PolynomialFeatures(2, interaction_only=True, include_bias=False) 
    X_inter = poly_features.fit_transform(X.as_gpu_matrix())
    X_df = cudf.DataFrame(X_inter, columns=poly_features.get_feature_names(X.columns))
    
    X_df = sm.tools.tools.add_constant(X_df.to_pandas())
    inter_model = linear_model.OLS(y.to_pandas().values, X_df).fit()
    if limit:
        inter_model = inter_model.pvalues[inter_model.pvalues < 0.05]
        print(inter_model.sort_values())
    else:
        print(inter_model.summary())
    
    return inter_model
    

## Single-Feature Predictions
Output is of the form: predictor, p-value

In [75]:
fit_model(df, 'xCO2_ATM_interpolated_ppm', True)

salinity_0      0.000052
salinity_2      0.001207
water_temp_8    0.008830
dtype: float64

In [76]:
fit_model(df, 'PRES_EQU_hPa', True)

salinity_0      0.000129
const           0.000152
salinity_2      0.001932
water_temp_8    0.013583
dtype: float64

In [77]:
fit_model(df, 'PRES_ATM@SSP_hPa', True)

salinity_0      0.000123
const           0.000205
salinity_2      0.001909
water_temp_8    0.012874
dtype: float64

In [78]:
fit_model(df, 'fCO2_SW@SST_uatm', True)

const           0.000059
salinity_0      0.000110
salinity_2      0.001892
water_temp_8    0.009946
dtype: float64

In [79]:
fit_model(df, 'fCO2_ATM_interpolated_uatm', True)

salinity_0      0.000046
salinity_2      0.001235
water_temp_8    0.007390
water_temp_6    0.048018
dtype: float64

In [80]:
fit_model(df, 'dfCO2_uatm', True)

const           3.307076e-22
salinity_0      4.021092e-04
salinity_2      3.139747e-03
water_temp_8    2.594908e-02
dtype: float64

## Interaction Columns

In [ ]:
df = df.dropna()

X = df[X_col_names]

equinox_col_names = dict({
    'xCO2_EQU_ppm': 'Mole fraction of CO2 in the equilibrator headspace (ppm)', 
    'xCO2_ATM_interpolated_ppm': 'Mole fraction of CO2 measured in dry outside air (ppm)',
    'PRES_EQU_hPa': 'Barometric pressure in the equilibrator headspace (hPa)', 
    'PRES_ATM@SSP_hPa':  'Barometric pressure measured outside, corrected to sea level (hPa)',
    'TEMP_EQU_C': 'Water temperature in equilibrator (°C)', 
    'SST_C': 'Sea surface temperature (°C)',
    'SAL_permil': 'Sea surface salinity on Practical Salinity Scale (ppt)',
    'fCO2_SW@SST_uatm': 'Fugacity of CO2 in sea water at SST and 100% humidity (μatm)',
    'fCO2_ATM_interpolated_uatm': 'Fugacity of CO2 in air corresponding to the interpolated xCO2 at SST and 100% humidity (μatm)',
    'dfCO2_uatm': 'Sea water fCO2 minus interpolated air fCO2 (μatm)' ,
    })


poly_features = PolynomialFeatures(2, interaction_only=True, include_bias=False) 
X_inter = poly_features.fit_transform(X.as_gpu_matrix())
X_inter_cols = poly_features.get_feature_names(X.columns)
X_df = cudf.DataFrame(X_inter, columns=poly_features.get_feature_names(X.columns))


def plot_correlation(df, hycom_col):
    graph_list = []
    
    data = pd.concat([X_df.to_pandas(), df[hycom_col].to_pandas()], axis=1)    
    hv_data = hv.Dataset(data)
    
    for idx, col in enumerate(X_inter_cols):
        scatter = hv.Scatter(hv_data, kdims=col, vdims=hycom_col).opts(width=600, height=400, title=equinox_col_names[hycom_col])
        scatter.redim(y=hv.Dimension(col))
        shaded = decimate(scatter, y_range=(0, data[hycom_col].max())).opts(width=600,  height=400)
        graph_list.append(shaded)
        del scatter
        
    layout = hv.Layout(graph_list).cols(2)
    return layout
    

plot_correlation(df, 'xCO2_EQU_ppm')

In [ ]:
plot_correlation(df, 'xCO2_ATM_interpolated_ppm')

In [ ]:
plot_correlation(df, 'dfCO2_uatm')

### Fitting Interaction Models


In [34]:
_ = fit_interaction_model(df, 'xCO2_ATM_interpolated_ppm', True)

salinity_2 water_temp_8      0.001458
water_temp_6 water_temp_8    0.001660
salinity_2 water_temp_6      0.002148
salinity_0 water_temp_8      0.002414
salinity_0 water_temp_6      0.004443
water_temp_4 water_temp_8    0.005247
salinity_4 water_temp_6      0.015091
water_temp_4 salinity_4      0.017746
salinity_2 water_temp_4      0.020974
water_temp_2 water_temp_8    0.021669
water_temp_4 salinity_6      0.029044
water_temp_6 salinity_6      0.038256
salinity_0 water_temp_4      0.039998
water_temp_4 water_temp_6    0.041155
dtype: float64


salinity_2 water_temp_8      0.001283
water_temp_6 water_temp_8    0.001733
salinity_2 water_temp_6      0.001861
salinity_0 water_temp_8      0.002447
salinity_0 water_temp_6      0.004198
water_temp_4 water_temp_8    0.005196
salinity_4 water_temp_6      0.013854
water_temp_4 salinity_4      0.017909
salinity_2 water_temp_4      0.018972
water_temp_2 water_temp_8    0.019375
water_temp_4 salinity_6      0.031389
salinity_0 water_temp_4      0.036999
water_temp_6 salinity_6      0.038100
water_temp_4 water_temp_6    0.039486
dtype: float64

In [20]:
fit_interaction_model(df, 'xCO2_ATM_interpolated_ppm')

salinity_0 water_temp_4      0.036999
salinity_0 water_temp_6      0.004198
salinity_0 water_temp_8      0.002447
water_temp_2 water_temp_8    0.019375
salinity_2 water_temp_4      0.018972
salinity_2 water_temp_6      0.001861
salinity_2 water_temp_8      0.001283
water_temp_4 salinity_4      0.017909
water_temp_4 water_temp_6    0.039486
water_temp_4 salinity_6      0.031389
water_temp_4 water_temp_8    0.005196
salinity_4 water_temp_6      0.013854
water_temp_6 salinity_6      0.038100
water_temp_6 water_temp_8    0.001733
dtype: float64


In [21]:
fit_interaction_model(df, 'PRES_EQU_hPa')

salinity_0 water_temp_4      0.037386
salinity_0 water_temp_6      0.004142
salinity_0 water_temp_8      0.002691
water_temp_2 water_temp_8    0.015803
salinity_2 water_temp_4      0.018860
salinity_2 water_temp_6      0.001803
salinity_2 water_temp_8      0.001470
water_temp_4 salinity_4      0.016572
water_temp_4 water_temp_6    0.027263
water_temp_4 salinity_6      0.027540
water_temp_4 water_temp_8    0.003239
salinity_4 water_temp_6      0.012239
water_temp_6 salinity_6      0.031526
water_temp_6 water_temp_8    0.001191
water_temp_6 salinity_8      0.046147
dtype: float64


In [22]:
fit_interaction_model(df, 'PRES_ATM@SSP_hPa')

salinity_0 water_temp_4      0.035087
salinity_0 water_temp_6      0.003845
salinity_0 water_temp_8      0.002567
water_temp_2 water_temp_8    0.015814
salinity_2 water_temp_4      0.017680
salinity_2 water_temp_6      0.001675
salinity_2 water_temp_8      0.001405
water_temp_4 salinity_4      0.016370
water_temp_4 water_temp_6    0.027350
water_temp_4 salinity_6      0.027910
water_temp_4 water_temp_8    0.003297
salinity_4 water_temp_6      0.012019
water_temp_6 salinity_6      0.031696
water_temp_6 water_temp_8    0.001221
water_temp_6 salinity_8      0.046371
dtype: float64


In [23]:
fit_interaction_model(df, 'TEMP_EQU_C')

salinity_0 water_temp_4      0.036941
salinity_0 water_temp_6      0.004324
salinity_0 water_temp_8      0.002817
water_temp_2 water_temp_8    0.018826
salinity_2 water_temp_4      0.017267
salinity_2 water_temp_6      0.001797
salinity_2 water_temp_8      0.001560
water_temp_4 salinity_4      0.014885
water_temp_4 water_temp_6    0.030125
water_temp_4 salinity_6      0.026782
water_temp_4 water_temp_8    0.004191
salinity_4 water_temp_6      0.011675
water_temp_6 salinity_6      0.031144
water_temp_6 water_temp_8    0.001704
dtype: float64


In [24]:
fit_interaction_model(df, 'SST_C')

salinity_0 water_temp_4      0.036855
salinity_0 water_temp_6      0.004312
salinity_0 water_temp_8      0.002810
water_temp_2 water_temp_8    0.018726
salinity_2 water_temp_4      0.017186
salinity_2 water_temp_6      0.001790
salinity_2 water_temp_8      0.001557
water_temp_4 salinity_4      0.014836
water_temp_4 water_temp_6    0.029824
water_temp_4 salinity_6      0.026764
water_temp_4 water_temp_8    0.004152
salinity_4 water_temp_6      0.011651
water_temp_6 salinity_6      0.031142
water_temp_6 water_temp_8    0.001698
dtype: float64


In [25]:
fit_interaction_model(df, 'SAL_permil')

salinity_0 water_temp_4      0.037343
salinity_0 water_temp_6      0.004565
salinity_0 water_temp_8      0.003030
water_temp_2 water_temp_8    0.020217
salinity_2 water_temp_4      0.017532
salinity_2 water_temp_6      0.001861
salinity_2 water_temp_8      0.001602
water_temp_4 salinity_4      0.014904
water_temp_4 water_temp_6    0.031622
water_temp_4 salinity_6      0.026517
water_temp_4 water_temp_8    0.004529
salinity_4 water_temp_6      0.011594
water_temp_6 salinity_6      0.030726
water_temp_6 water_temp_8    0.001876
dtype: float64


In [103]:
_ = fit_interaction_model(df, 'fCO2_SW@SST_uatm', True)

water_temp_6 water_temp_8    0.000394
water_temp_4 water_temp_8    0.002606
salinity_2 water_temp_6      0.002898
salinity_2 water_temp_8      0.003278
salinity_0 water_temp_8      0.005696
salinity_0 water_temp_6      0.006479
salinity_4 water_temp_6      0.012539
water_temp_4 salinity_4      0.014885
const                        0.021036
salinity_2 water_temp_4      0.021969
water_temp_4 salinity_6      0.024277
water_temp_6 salinity_6      0.027231
water_temp_2 water_temp_8    0.032403
water_temp_4 water_temp_6    0.037562
salinity_0 water_temp_4      0.045051
water_temp_6 salinity_8      0.049619
dtype: float64


In [23]:
_ = fit_interaction_model(df, 'fCO2_ATM_interpolated_uatm')

salinity_2 water_temp_8      0.001215
water_temp_6 water_temp_8    0.001495
salinity_2 water_temp_6      0.001782
salinity_0 water_temp_8      0.002363
salinity_0 water_temp_6      0.003946
water_temp_4 water_temp_8    0.004597
salinity_4 water_temp_6      0.014026
water_temp_2 water_temp_8    0.017428
water_temp_4 salinity_4      0.019022
salinity_2 water_temp_4      0.019203
water_temp_4 salinity_6      0.032619
salinity_0 water_temp_4      0.035973
water_temp_4 water_temp_6    0.037050
water_temp_6 salinity_6      0.038566
dtype: float64


In [47]:
res = fit_interaction_model(df, 'dfCO2_uatm', True)

water_temp_6 water_temp_8    0.000286
water_temp_4 water_temp_8    0.001925
salinity_2 water_temp_6      0.003208
salinity_2 water_temp_8      0.005467
salinity_0 water_temp_6      0.008277
salinity_4 water_temp_6      0.009406
salinity_0 water_temp_8      0.009426
water_temp_4 salinity_4      0.010615
water_temp_4 salinity_6      0.018773
water_temp_6 salinity_6      0.018986
salinity_2 water_temp_4      0.019393
water_temp_4 water_temp_6    0.030431
water_temp_8 salinity_8      0.040806
water_temp_2 water_temp_8    0.041789
water_temp_6 salinity_8      0.046094
salinity_0 water_temp_4      0.047530
const                        0.049034
dtype: float64


## Feature Subset Evaluation
Predictors with lowest p-values for fCO2_SW@SST_uatm selected below, and model performance evaluated using cross validation

In [176]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import Ridge
import cuml
from cuml.experimental.preprocessing import PolynomialFeatures

from cuml.linear_model import Lasso
X_col_subset = [
    'salinity_0',
    'salinity_2',
    'water_temp_8',
    'water_temp_6 water_temp_8',   
    'salinity_2 water_temp_6',      
    'water_temp_4 water_temp_8',    
    'salinity_2 water_temp_8',      
    'salinity_0 water_temp_8',      
    'salinity_0 water_temp_6',      
    'salinity_4 water_temp_6',      
    'water_temp_4 salinity_4',      
    'salinity_2 water_temp_4',      
    'water_temp_4 salinity_6',      
    'water_temp_6 salinity_6',      
    'water_temp_2 water_temp_8',    
    'water_temp_4 water_temp_6',    
    'salinity_0 water_temp_4',          
    ]
    

pred_col = 'fCO2_SW@SST_uatm'
df = df.dropna()

df = df[df[pred_col] >= 0]

X = df[X_col_names]
y = df[[pred_col]]

poly_features = PolynomialFeatures(2, interaction_only=True, include_bias=False) 
X_inter = poly_features.fit_transform(X)

X_df = X_inter
X_df.columns = poly_features.get_feature_names(X.columns)    
X_df = X_df[X_col_subset]

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

model = Lasso(alpha = 0.1) 
model.fit(X_train, y_train)

Lasso()

In [177]:
!nvidia-smi

Mon Jul  5 14:20:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   70C    P2    30W /  N/A |   2054MiB /  6078MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [178]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test.to_pandas(), y_pred.to_pandas())
print(mae)

6.744775374498608
